In [2]:
import os
import numpy as np
from osgeo import gdal, osr

def split_tif(folder_path, file_name, block_size):
    # Ensure the output directory exists
    out_folder = os.path.join(folder_path, f"{block_size}m_{block_size}m")
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)

    # Open the original TIFF file
    tif_file = gdal.Open(file_name)

    if tif_file is None:
        print(f"Error: Failed to open file {file_name}")
        return

    # Get the geotransform information
    geotransform = tif_file.GetGeoTransform()
    xmin = geotransform[0]
    ymax = geotransform[3]
    res = geotransform[1]

    # Get the raster size
    x_size = tif_file.RasterXSize
    y_size = tif_file.RasterYSize

    # Iterate over the bounding boxes of each block
    for y in np.arange(ymax, ymax - y_size * res, -block_size):
        for x in np.arange(xmin, xmin + x_size * res, block_size):
            # Calculate the coordinates of the block
            x_min_block, y_max_block = x, y
            x_max_block, y_min_block = x + block_size, y - block_size
            block_coord = [x + block_size / 2, y - block_size / 2]

            # Read the pixel values for the block
            pixel_values = tif_file.ReadAsArray(
                int((x - xmin) / res), int((ymax - y) / res), int(block_size / res), int(block_size / res))

            if pixel_values is None:
                print("Error: Failed to read pixel values.")
                continue  # Skip processing this block and continue to the next one

            # Check if the alpha band exists and if any pixel value is 0
            alpha_band_index = tif_file.RasterCount - 1  # Assuming the alpha band is the last band
            alpha_band_values = pixel_values[alpha_band_index]

            if np.any(alpha_band_values == 0):
                print(f"Skipping block at {block_coord} due to alpha band value of 0.")
                continue  # Skip this block if any pixel in the alpha band is 0

            # Create a new dataset for the block
            driver = gdal.GetDriverByName('GTiff')

            if driver is None:
                print("Error: Failed to get GDAL driver.")
                exit(1)

            out_tif = driver.Create(
                os.path.join(out_folder, f"{int(block_coord[0])}_{int(block_coord[1])}.tif"),
                int(block_size / res), int(block_size / res), tif_file.RasterCount, gdal.GDT_Byte
            )

            if out_tif is None:
                print("Error: Failed to create output TIFF dataset.")
                continue  # Skip processing this block and continue to the next one

            # Set the geotransform
            out_tif.SetGeoTransform((x_min_block, res, 0, y_max_block, 0, -res))
            srs = osr.SpatialReference()
            srs.ImportFromEPSG(3826)  # TWD97 lat/long
            out_tif.SetProjection(srs.ExportToWkt())

            # Write the pixel values to the new dataset
            for band_num in range(tif_file.RasterCount):
                band_values = pixel_values[band_num]
                out_tif.GetRasterBand(band_num + 1).WriteArray(band_values)

            # Write to disk
            out_tif.FlushCache()

            # Close the new dataset
            out_tif = None

    # Close the original TIFF file
    tif_file = None



In [3]:
# Example usage
folder_path = "H:\\Yehmh\\ZF\\202307"
file_name = "H:\\Yehmh\\ZF\\202307\\M3M202307_51_transparent_mosaic_group1.tif"
split_tif(folder_path, file_name, 5)

Skipping block at [299968.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [299973.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [299978.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [299983.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [299988.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [299993.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [299998.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [300003.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [300008.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [300013.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [300018.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [300023.44832, 2632803.57816] due to alpha band value of 0.
Skipping block at [300028.44832, 2632803.57816] due to alpha ban